In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import itertools
import warnings
warnings.filterwarnings('ignore')

In [2]:
# For Laptop

filename2 = '/home/jovyan/work/data/revolut/rev-notifications.csv'
rev_notifications = pd.read_csv(filename2)
df_rev_notifications = rev_notifications.copy()


# DATA PREPROCESSING

In [3]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
    return mis_val_table_ren_columns

In [4]:
def unqiue_values_table(df):
    unq_val = df.nunique()
    unq_val_percent = 100 * df.nunique() / len(df)
    unq_val_table = pd.concat([unq_val, unq_val_percent], axis=1)
    unq_val_table_ren_columns = unq_val_table.rename(
    columns = {0 : 'Unique Values', 1 : '% of Total Values'})
    unq_val_table_ren_columns = unq_val_table_ren_columns[
        unq_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    'Unique Values', ascending=False).round(2)
    print ("Your selected dataframe has " + str(df.shape[0]) + " rows "       
        "and " + str(df.shape[1]) + " columns.")
    return unq_val_table_ren_columns

In [5]:
def date_conversion(df):
    df['created_date'] = pd.to_datetime(df['created_date'])
    return df

### NOTIFICATIONS 

In [6]:
missing_values_table(df_rev_notifications)

Your selected dataframe has 5 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [7]:
unqiue_values_table(df_rev_notifications)

Your selected dataframe has 121813 rows and 5 columns.


,Unique Values,% of Total Values
created_date,121810,100.00
user_id,18953,15.56
reason,17,0.01
channel,3,0.00
status,2,0.00


In [8]:
df_rev_notifications.status.unique()

array(['SENT', 'FAILED'], dtype=object)

In [9]:
date_conversion(df_rev_notifications)

,reason,channel,status,user_id,created_date
0,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_7086,2018-12-02 17:58:33.320645
1,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_6598,2018-12-01 23:09:37.367127
2,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_4151,2018-12-04 02:57:56.425660
3,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_1408,2018-12-11 02:04:46.284683
4,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_6292,2018-12-14 17:09:58.900808
...,...,...,...,...,...
121808,LOST_CARD_ORDER,EMAIL,SENT,user_12687,2019-05-07 12:26:28.185772
121809,LOST_CARD_ORDER,EMAIL,SENT,user_16875,2019-05-10 01:25:42.530159
121810,LOST_CARD_ORDER,EMAIL,SENT,user_17426,2019-05-09 08:42:00.245644
121811,LOST_CARD_ORDER,EMAIL,SENT,user_15880,2019-05-10 23:01:30.998474


##### How many days in between notifications 

In [10]:
df = df_rev_notifications.loc[df_rev_notifications.status.isin(['SENT']),['user_id','created_date','channel']]
df = df.sort_values(["user_id", "created_date"], ascending = (True, True))
df['diff_in_date'] = df.groupby('user_id')['created_date'].diff().dt.days
df.fillna(0,inplace = True)

def notification_flag(x):
    if pd.isna(x):
        return 0
    else :
        return 1
    
df['count_ntf']= df['created_date'].apply(notification_flag).astype(float)

df['year']= df['created_date'].dt.year
df['month']= df['created_date'].dt.month
#temp = df.groupby(['user_id','year','month']).agg({'diff_in_date':'mean'})
df = df.groupby(['user_id','year','channel']).agg({'diff_in_date':'mean', 'count_ntf':'sum'}).reset_index()


In [11]:
tip_us_gb = pd.pivot_table(df, values = ['count_ntf','diff_in_date'], index=['user_id','year'], columns = ['channel']).reset_index()
tip_us_copy = tip_us_gb.copy()
tip_us_copy.columns = ['_'.join(col).strip()  for col in tip_us_gb.columns.values]
tip_us_copy.reset_index(drop=True, inplace=True)
tip_us_copy.reset_index(drop=True, inplace=True)
missing_values_table(tip_us_copy)



Your selected dataframe has 8 columns.
There are 6 columns that have missing values.


,Missing Values,% of Total Values
count_ntf_SMS,30790,98.5
diff_in_date_SMS,30790,98.5
count_ntf_PUSH,8533,27.3
diff_in_date_PUSH,8533,27.3
count_ntf_EMAIL,7105,22.7
diff_in_date_EMAIL,7105,22.7


In [12]:
tip_us_copy.drop(columns = ['count_ntf_SMS','diff_in_date_SMS'],inplace = True)
tip_us_copy.fillna(0,inplace=True)
tip_us_copy

,user_id_,year_,count_ntf_EMAIL,count_ntf_PUSH,diff_in_date_EMAIL,diff_in_date_PUSH
0,user_0,2018,0.0,3.0,0.0,13.000000
1,user_0,2019,2.0,3.0,16.0,14.333333
2,user_1,2018,1.0,0.0,0.0,0.000000
3,user_1,2019,1.0,0.0,384.0,0.000000
4,user_10,2018,0.0,1.0,0.0,0.000000
...,...,...,...,...,...,...
31258,user_9997,2018,0.0,3.0,0.0,18.666667
31259,user_9997,2019,0.0,1.0,0.0,14.000000
31260,user_9998,2019,1.0,0.0,0.0,0.000000
31261,user_9999,2018,0.0,2.0,0.0,15.500000
